In [ ]:
# download_stopwords()
# download_lemmatizer()


In [7]:
import os

train_pos = [fname.split('.')[0] for fname in os.listdir('./data/aclImdb/train/pos')]
test_pos = [fname.split('.')[0] for fname in os.listdir('./data/aclImdb/test/pos')]
train_neg = [fname.split('.')[0] for fname in os.listdir('./data/aclImdb/train/neg')]
test_neg = [fname.split('.')[0] for fname in os.listdir('./data/aclImdb/test/neg')]

positive = train_pos
positive.extend(test_pos)
negative = train_neg
negative.extend(test_neg)
grades = [fname.split('_')[1] for fname in positive]
grades.extend([fname.split('_')[1] for fname in negative])

grade_cnt = {}
for grade in grades:
	grade_cnt[grade] = grade_cnt.get(grade, 0) + 1

for key in grade_cnt.keys():
	print(f"{key}: {grade_cnt[key]}")

9: 4607
8: 5859
10: 9731
7: 4803
3: 4961
4: 5331
1: 10122
2: 4586


In [2]:
from main import main

N = 25000
N_UNIGRAMS = 2500

TRAIN_SIZE = N*3//4
TEST_SIZE = N//4

# REPS = 5

# for i in range(REPS):
# 	print(str(i+1)+":")
# await main(N_UNIGRAMS, TRAIN_SIZE, TEST_SIZE, True, use_linear_svm=False)
# print()
await main(N_UNIGRAMS, TRAIN_SIZE, TEST_SIZE, True, use_linear_svm=True)

NaiveBayes accuracy: 82.64% 10330/12500
SVM accuracy: 87.848% 10981/12500
loading: 661.2575903999386
processing: 74.8712935000658s
NB training: 3.0790324999252334s
SVM training: 1.4196735000004992s
